## import

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install xgboost
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

## input dataset

In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_male.txt', delimiter='\t')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 7 to 461
Columns: 297 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(141), int64(156)
memory usage: 237.5 KB
None


In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_male.txt', delimiter='\t')
data=df.iloc[:,1:297]
target=df.iloc[:,0]
target= target.replace(4,0)
target= target.replace(5,1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 7 to 461
Columns: 297 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(141), int64(156)
memory usage: 237.5 KB
None


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 7 to 461
Columns: 296 entries, MANIC_MOOD_EVER to A8d_.1.tell_professional
dtypes: float64(141), int64(155)
memory usage: 236.7 KB
None


## xgboost

In [ ]:
# Initialize an array to store feature importance for each iteration
import numpy as np
feature_importances = np.zeros((50, len(data.columns)))

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# repeat 50 times
for i in range(50):
    train_x, test_x, train_y, test_y = train_test_split(data,target,test_size=0.2,random_state=(i*i+5))
    model = XGBClassifier(
        n_estimators=25,
        max_depth=5,
        learning_rate= 1,
        subsample=0.75,
        colsample_bytree=0.75,
        reg_lambda=10,
        eval_metric='auc',
        early_stopping_rounds=5,
        random_state=i
    )
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)])
    #perm = PermutationImportance(model, random_state=101).fit(test_x, test_y)
    feature_importances[i, :] = model.feature_importances_

[0]	validation_0-auc:0.48980
[1]	validation_0-auc:0.64286
[2]	validation_0-auc:0.76020
[3]	validation_0-auc:0.83673
[4]	validation_0-auc:0.82653
[5]	validation_0-auc:0.79592
[6]	validation_0-auc:0.83673
[7]	validation_0-auc:0.81633
[8]	validation_0-auc:0.81633
[0]	validation_0-auc:0.83333
[1]	validation_0-auc:0.77778
[2]	validation_0-auc:0.74444
[3]	validation_0-auc:0.75000
[4]	validation_0-auc:0.78333
[5]	validation_0-auc:0.77222
[0]	validation_0-auc:0.63750
[1]	validation_0-auc:0.73125
[2]	validation_0-auc:0.77500
[3]	validation_0-auc:0.70000
[4]	validation_0-auc:0.72500
[5]	validation_0-auc:0.75000
[6]	validation_0-auc:0.72500
[7]	validation_0-auc:0.73750
[0]	validation_0-auc:0.56250
[1]	validation_0-auc:0.65625
[2]	validation_0-auc:0.76875
[3]	validation_0-auc:0.73125
[4]	validation_0-auc:0.78750
[5]	validation_0-auc:0.81250
[6]	validation_0-auc:0.78750
[7]	validation_0-auc:0.76250
[8]	validation_0-auc:0.75000
[9]	validation_0-auc:0.78750
[0]	validation_0-auc:0.77206
[1]	validation

## permutation importance

# #result

In [ ]:
# Calculate average feature importance
average_importances = np.mean(feature_importances, axis=0)

# Sort and get the index of the top 5 features
top_10_feature_indices = np.argsort(average_importances)[-10:]

# Get and print the top 5 most important feature names and their average importance
top_features = data.columns[top_10_feature_indices]
for feature, importance in zip(top_features, average_importances[top_10_feature_indices]):
    print(f"Feature: {feature}, Importance: {importance}")

Feature: HALLUC_LEVEL, Importance: 0.01910618045250885
Feature: Factor.4., Importance: 0.01999983191024512
Feature: Emot_Abuse, Importance: 0.021288315318524837
Feature: Bmi_Current, Importance: 0.021533713799435646
Feature: Alc_Dependence, Importance: 0.03523001654073596
Feature: Ppwork_12work_37not, Importance: 0.05056708674877882
Feature: Neglect, Importance: 0.05344852177426219
Feature: ALCOHOL_PROBLEM, Importance: 0.057019864171743394
Feature: STIM_LEVEL, Importance: 0.0622990248631686
Feature: PHQ_7, Importance: 0.1140346285328269
